In [ ]:

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pyautogui as pg
import time
from tqdm import tqdm
import pandas as pd
from config import *


def create_list():
    # create a recipient list for the clinic
    try:
        # find the most common element we want to add
        checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='ref_check y-email y-mailer']")
        checkbox.click()
        time.sleep(.5)
        # website nav
        bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
        bulk_dropdown.click()
        time.sleep(.5)
        create_list = driver.find_element(By.ID, 'create_actives_email_list')
        create_list.click()
        
        time.sleep(.5)
        # because of the website architecture, selenium had trouble grabbing the iframe and overlays consistently,
        # so we will use computer vision to navigate correctly every time
        create_list = pg.locateCenterOnScreen('resources/create_email_list.png',confidence=.8)
        pg.click(create_list[0], create_list[1])
        pg.typewrite('Primary Email List - January 2023')
        # button is a different color from everything else on screen, but we want it to be clicks 10/10 times and .6
        # is a good lower confidence threshold for the use case
        finalize_list = pg.locateCenterOnScreen('resources/finalize.png', confidence=.6)
        pg.click(finalize_list[0], finalize_list[1])
        pg.click(finalize_list[0], finalize_list[1])             
        pg.click(finalize_list[0], finalize_list[1])             

    except:
        # alternative checkbox ( <5% occurence )
        checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='n-mailer ref_check y-email']")
        checkbox.click()
        time.sleep(.5)
        # navigate
        bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
        bulk_dropdown.click()
        time.sleep(.5)
        create_list = driver.find_element(By.ID, 'create_actives_email_list')
        create_list.click()
        
        time.sleep(.5)

        # using cv2 via pyautogui to locate and click on page resources
        create_list = pg.locateCenterOnScreen('resources/create_email_list.png',confidence=.8)
        pg.click(create_list[0], create_list[1])
        pg.typewrite('Primary Email List - January 2023')
        finalize_list = pg.locateCenterOnScreen('resources/finalize.png', confidence=.6)
        pg.click(finalize_list[0], finalize_list[1])             
        pg.click(finalize_list[0], finalize_list[1])             
        pg.click(finalize_list[0], finalize_list[1])
    time.sleep(.5)
    # if the email list already exists, move on (the first person has already been added in this case)
    already_exists = pg.locateCenterOnScreen('resources/fail_to_create.png', confidence=.5)
    if already_exists:
        pg.press('esc')
        pg.press('esc')
        time.sleep(.5)
                     
def add_to_list():
        try:
            # similar layout and reasoning as create_list()
            checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='ref_check y-email y-mailer']")
            checkbox.click()
            time.sleep(.5)
            bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
            bulk_dropdown.click()

            time.sleep(.5)

            add_to_email = driver.find_element(By.ID, 'add_actives_to_email_list')
            add_to_email.click()

            time.sleep(.5)
            # if the list is already selected, click button
            list_added = pg.locateCenterOnScreen('resources/correct_list.png', confidence=.8)
            if list_added:
                finalize_list = pg.locateCenterOnScreen('resources/add.png',confidence=.8)
                pg.click(finalize_list[0],finalize_list[1])
                # pg.click(finalize_list[0],finalize_list[1])
            # otherwise navigate to the dropdown, and select the correct list
            else:
                # unique element is not the place we need to click, so we adjust the x and y  to match.
                # if working with multiple monitors of differing size these values can be changed to % of window frame
                add_overlay = pg.locateCenterOnScreen('resources/add_email.png',confidence=.8)
                pg.click(add_overlay[0]+50 ,add_overlay[1]+87 )
                time.sleep(.5)
                
                # click the button
                choose_campaign = pg.locateCenterOnScreen('resources/email_pick.png',confidence=.8)
                pg.click(choose_campaign[0],choose_campaign[1])
                time.sleep(.5)
            
            finalize_list = pg.locateCenterOnScreen('resources/add.png',confidence=.8)
            pg.click(finalize_list[0],finalize_list[1])
            # pg.click(finalize_list[0],finalize_list[1])
                
        except:   
            try: 
                # same as above, for the corner case where we have email info, but not mailing info
                checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='n-mailer ref_check y-email']")
                checkbox.click()
                bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
                bulk_dropdown.click()

                time.sleep(.5)

                add_to_email = driver.find_element(By.ID, 'add_actives_to_email_list')
                add_to_email.click()

                time.sleep(.5)

                list_added = pg.locateCenterOnScreen('resources/correct_list.png', confidence=.8)
                if list_added:
                    finalize_list = pg.locateCenterOnScreen('resources/add.png',confidence=.8)
                    pg.click(finalize_list[0],finalize_list[1])
                    # pg.click(finalize_list[0],finalize_list[1])
                else:    
                    add_overlay = pg.locateCenterOnScreen('resources/add_email.png',confidence=.8)
                    pg.click(add_overlay[0]+50 ,add_overlay[1]+87 )
                    time.sleep(.5)
                    
                    choose_campaign = pg.locateCenterOnScreen('resources/email_pick.png',confidence=.8)
                    pg.click(choose_campaign[0],choose_campaign[1])
                    time.sleep(.5)
                    finalize_list = pg.locateCenterOnScreen('resources/add.png',confidence=.8)
                    pg.click(finalize_list[0],finalize_list[1])
                    # pg.click(finalize_list[0],finalize_list[1])
                    time.sleep(.5)

            except:
                pass              

def actives(i):
    # source list csv/folder not included in public repo
    try:
        # majority of cases
        actives = pd.read_csv(f'primary_emails_only/{i[11:]} active-email.csv')
    except:
        try:
            # edge cases
            actives = pd.read_csv(f'primary_emails_only/{i[10:]} active-email.csv')
        except:
            # there are a few that had to be done by hand, including one that had a space as its first character.
            pass
    # format the name based on columns
    names = list(actives['First Name'] + ' ' + actives['Last Name'])
    # return the list of names
    return names      

#standard selenium set up. 
# url is defined in config along with u(sername) and pw (password)
response = requests.get(url)
chrome_options = webdriver.ChromeOptions()
# there is a pop up that covers part of the dropdown in a default opening.
chrome_options.add_argument('--start-maximized')

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

username = driver.find_element( By.ID, 'user_email')
password = driver.find_element(By.ID, 'user_password')

# enter the login credentials
username.send_keys(u)
password.send_keys(pw)

login_button = driver.find_element(By.NAME, 'commit')
login_button.click()

# navigate to the main dropdown
dropdown = driver.find_element(By.ID, 'user-usurp')
dropdown.click()

# create a list of clickable elements in the dropdown
options = dropdown.find_elements(By.PARTIAL_LINK_TEXT, 'Therapydia')

link_texts = []
# grab the link text from each of the dropdown clickable elements
for option in options:
    text = option.text
    link_texts.append(text)

# removes headquarters from list
link_texts = link_texts[1:]

# this goes backwards through the locations, to hit corner cases quicker and trial more effectively
for i in link_texts[::-1]:
    # this try except is for cases where the server is slower to load links, or a pop up is blocking the next element.
    try:
        location = driver.find_element(By.LINK_TEXT, i)
        location.click()    
    #this except block was never called during full launch, but left in for the next time it runs
    except:
        time.sleep(10)
        location = driver.find_element(By.LINK_TEXT, i)
        location.click()    
    
    try:
        # printing the location name to be able to track where we are in the process and extract our location specific recipient list
        print(f'{i}:')
        names = actives(i)

        # navigate, and wait to load
        sources = driver.find_element(By.LINK_TEXT, 'Sources')
        sources.click()
        time.sleep(.5)

        # progress bar. its a 2 hour process
        for g in tqdm(names):
            # enter name into search box
            search_referrer = driver.find_element(By.NAME, 'search_referrer')
            search_referrer.send_keys(g)
            # does not search dynamically
            pg.press('enter')
            time.sleep(.5)
            # creates the new list if this is the first name in the searched list.
            if g == names[0]:
                create_list()
            else:
            # adds to the list if not the first name in the search list    
                add_to_list()
            search_referrer.clear()
        # the website is slow to load, we wait
        time.sleep(2)
        # and go next
        dropdown = driver.find_element(By.CLASS_NAME, "dropdown-toggle")
        dropdown.click()
        time.sleep(3)
    except:
        # no list downloaded = skip
        print(f'{i} has no relevant sources')
        dropdown = driver.find_element(By.CLASS_NAME, "dropdown-toggle")
        dropdown.click()
        time.sleep(3)

